In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import os
from nltk.tokenize import sent_tokenize
import pandas as pd
from wordcloud import WordCloud
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random
from nltk.corpus import stopwords
from data import reduce_genres

Using TensorFlow backend.


 Dataset :: namedtuple(
  'info': pandas.df
  'labels': pandas.df('filename.txt': 'genre')
  'genres': ['genre'] # unique genres
  'label_dataset': SubDataset
  'sentiment_dataset': SubDataset
  'book_sentiment_words_list': ['filename']

 SubDataset :: namedtuple(
   'dict_index_to_label' = dict to convert label_index -> label_name
   'dict_label_to_index'= dict to convert label_name -> label_index



In [2]:
import config
import tfidf2 as tfidf
os.getcwd()

'/Users/Tristan/books/src'

In [3]:
os.listdir(config.dataset_dir)

['labels.csv',
 'bookdata_alt.csv',
 '.DS_Store',
 'bookdata.csv',
 'test',
 'final_data.csv',
 'bookdatabase',
 'output',
 'Open_Sans_Condensed',
 'unique_genres.txt',
 'results_file.txt']

In [4]:
def make_lex_dict(lexicon_file):
        """
        Convert lexicon file to a dictionary
        """
        lex_dict = {}
        for line in lexicon_file.split('\n'):
            (word, measure) = line.strip().split('\t')[0:2]
            lex_dict[word] = float(measure)
        return lex_dict
    
sent_dict = make_lex_dict(open('/Users/Tristan/books/src/' +'vader_lexicon.txt', 'r').read())


Sentiment analysis. Analysis is performed for each sentence and the sentiment scores kept in lists. Sentiment scores are calculated by averaging the sentiment scores for all sentences.

In [5]:
def return_sentiment_scores(sentence):
    # return just the sentiment scores
    snt = analyser.polarity_scores(sentence)
    return snt

def sentiment_analysis(directory):
    analyser = SentimentIntensityAnalyzer()                    
    # returns the sentiment of every book in the directory
    data = pd.read_csv(config.dataset_dir + 'output/final_data.csv', index_col=0)
    print(len(data.index))
#     max_amt = len(data.index) + 2
#     print(data.index, len(os.listdir(directory)))
    pos_list = []
    neg_list = []
    neu_list = []
    comp_list = []
    
    # for every book
    for filename in data['filename']:#[:max_amt]:
        
        sub_pos_list = []
        sub_neg_list = []
        sub_neu_list = []
        sub_comp_list = []
        
        # if file is a textfile
        if filename.endswith(".txt"):
            text = open(os.path.join(directory, filename), 'r', errors='replace')
            # for every line in the text
            for line in text.readlines():
                scores = return_sentiment_scores(line)
                # save sentiment scores 
                sub_neg_list.append(scores['neg'])
                sub_neu_list.append(scores['neu'])
                sub_pos_list.append(scores['pos'])
                sub_comp_list.append(scores['compound'])
            
            # then save average sentiment scores for each book
            neg_list.append((sum(sub_neg_list) / float(len(sub_neg_list))))
            pos_list.append((sum(sub_pos_list) / float(len(sub_pos_list))))
            neu_list.append((sum(sub_neu_list) / float(len(sub_neu_list))))
            comp_list.append((sum(sub_comp_list) / float(len(sub_comp_list))))
            
    # convert scores to pandas compatible list
    neg = pd.Series(neg_list)
    pos = pd.Series(pos_list)
    neu = pd.Series(neu_list)
    com = pd.Series(comp_list)

    print(len(neg), len(pos), len(neu), len(com))
    # fill the right columns with the right data
    print(type(data),'type')
    print(neg)
    data['neg score'] = neg.values
    data['pos score'] = pos.values
    data['neu score'] = neu.values
    data['comp score'] = com.values
    data.to_csv(config.dataset_dir + 'output/final_data.csv')
    return data

In [6]:
analyser = SentimentIntensityAnalyzer()                    
sentiment_analysis(config.dataset_dir + 'bookdatabase/books/')

915
915 915 915 915
<class 'pandas.core.frame.DataFrame'> type
0      0.064028
1      0.044295
2      0.049711
3      0.047641
4      0.061587
5      0.048907
6      0.057010
7      0.045972
8      0.054442
9      0.037464
10     0.037340
11     0.055954
12     0.041392
13     0.055000
14     0.068802
15     0.052302
16     0.041800
17     0.043438
18     0.026211
19     0.056150
20     0.044526
21     0.034635
22     0.033165
23     0.040971
24     0.054841
25     0.048444
26     0.058575
27     0.052053
28     0.078131
29     0.045618
         ...   
885    0.040172
886    0.060173
887    0.017144
888    0.038590
889    0.044522
890    0.057267
891    0.061412
892    0.046271
893    0.057172
894    0.054442
895    0.061912
896    0.056598
897    0.051939
898    0.057441
899    0.038862
900    0.032124
901    0.051378
902    0.073029
903    0.043030
904    0.064964
905    0.048816
906    0.043717
907    0.055604
908    0.029775
909    0.035116
910    0.050853
911    0.053077
912    0.

,filename,title.1,author,release year,genre,pos score,neg score,neu score,comp score
title,,,,,,,,,
Jonah,3678.txt,Jonah,Louis Stone,"January, 2003",poetri,0.063752,0.064028,0.670289,0.001590
King--of the Khyber Rifles,6066.txt,King--of the Khyber Rifles,Talbot Mundy,"July, 2004",adventur,0.056072,0.044295,0.660100,0.020974
The Wind in the Willows,289.txt,The Wind in the Willows,Kenneth Grahame,"July, 1995",children,0.075622,0.049711,0.708466,0.050425
Akbar,40155-8.txt,Akbar,P. A. S. van Limburg-Brouwer,"July 7, 2012",syair,0.088927,0.047641,0.701412,0.078233
Erewhon,1906.txt,Erewhon,Samuel Butler,"March 20, 2005",satir,0.075006,0.061587,0.750951,0.025886
Night and Day,1245.txt,Night and Day,Virginia Woolf,"March, 1998",romance,0.070135,0.048907,0.711205,0.043948
She and Allan,5745.txt,She and Allan,H. Rider Haggard,"April 22, 2006",fantasi,0.067608,0.057010,0.693447,0.020357
The Amateur Cracksman,706.txt,The Amateur Cracksman,E. W. Hornung,"November, 1996",crime,0.059576,0.045972,0.663071,0.026517
Anne of Geierstein,44247-8.txt,Anne of Geierstein,Walter Scott,"November 21, 2013",histor,0.080691,0.054442,0.716062,0.048409


We also want to count the amount of positive and negative words as features. We also create a new file for each book with just the sentiment words. As a result, we will be able to do tfidf on these files later and create wordclouds per genre.

In [7]:
def count_sentiment_words(directory):
    sent_words_list =[]
    pos_list = []
    neg_list = []
    
    data = pd.read_csv(config.dataset_dir + 'output/final_data.csv', index_col=0)

    for filename in data['filename']:
        sent_words_list =[]
        pos_count = 0
        neg_count = 0
        
        if filename.endswith(".txt"):
            text = open(os.path.join(directory, filename), 'r', errors='replace')
            sentiment_file = open(config.dataset_dir +'output/sentiment_word_texts/' + filename , 'w')

            for line in text.readlines():
                for word in line.split(" "):
                    if word in sent_dict:
                        if sent_dict[word] >= 0:
                            pos_count += 1
                            sent_words_list.append(word)
                            sentiment_file.write("%s" % word)
                            sentiment_file.write(" ")
                        else:
                            neg_count += 1
                            sentiment_file.write("%s" % word)
                            sentiment_file.write(" ")

            pos_list.append(pos_count)
            neg_list.append(neg_count)
            
    data['amt pos'] = pos_list 
    data['amt neg'] = neg_list
    
    data.to_csv(config.dataset_dir + 'output/final_data.csv')
    return data

count_sentiment_words(config.dataset_dir + 'bookdatabase/books/')


,filename,title.1,author,release year,genre,pos score,neg score,neu score,comp score,amt pos,amt neg
title,,,,,,,,,,,
Jonah,3678.txt,Jonah,Louis Stone,"January, 2003",poetri,0.063752,0.064028,0.670289,0.001590,2058,2017
King--of the Khyber Rifles,6066.txt,King--of the Khyber Rifles,Talbot Mundy,"July, 2004",adventur,0.056072,0.044295,0.660100,0.020974,2518,2115
The Wind in the Willows,289.txt,The Wind in the Willows,Kenneth Grahame,"July, 1995",children,0.075622,0.049711,0.708466,0.050425,1684,1172
Akbar,40155-8.txt,Akbar,P. A. S. van Limburg-Brouwer,"July 7, 2012",syair,0.088927,0.047641,0.701412,0.078233,2988,1652
Erewhon,1906.txt,Erewhon,Samuel Butler,"March 20, 2005",satir,0.075006,0.061587,0.750951,0.025886,2506,1952
Night and Day,1245.txt,Night and Day,Virginia Woolf,"March, 1998",romance,0.070135,0.048907,0.711205,0.043948,4822,3088
She and Allan,5745.txt,She and Allan,H. Rider Haggard,"April 22, 2006",fantasi,0.067608,0.057010,0.693447,0.020357,3234,2652
The Amateur Cracksman,706.txt,The Amateur Cracksman,E. W. Hornung,"November, 1996",crime,0.059576,0.045972,0.663071,0.026517,1363,1072
Anne of Geierstein,44247-8.txt,Anne of Geierstein,Walter Scott,"November 21, 2013",histor,0.080691,0.054442,0.716062,0.048409,3268,2205


In [8]:
import pandas as pd

In [9]:
def read_unique_genres():
    genres_file = open(config.dataset_dir + 'unique_genres.txt', 'r')
    return[genre.strip('\n') for genre in genres_file.readlines()]

In [11]:
def create_wordcloud(scores, genre):
    
    font_path = config.dataset_dir + 'Open_Sans_Condensed/OpenSansCondensed-Light.ttf'
    stopWords = set(stopwords.words('english'))

    try:
        w = WordCloud(stopwords = stopWords, background_color='white', min_font_size=14, font_path=font_path, width = 1000, height = 500,relative_scaling=1,normalize_plurals=False)
        wordcloud = w.generate_from_frequencies(scores)
        wordcloud.recolor(color_func=grey_color_func)

        
    except ZeroDivisionError:
        print('shit')
        return
    
    plt.figure(figsize=(15,8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.savefig(config.dataset_dir + 'output/wordclouds/' + genre + '.png')
    plt.close()
    
def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 0%%, %d%%)" % random.randint(10, 50)


def tfidf_per_genre(plot_wc=False):
    data = pd.read_csv(config.dataset_dir + 'output/final_data.csv')
    genres_file = open(config.dataset_dir + 'unique_genres.txt', 'r')
    pre_genre_list = [genre.strip('\n') for genre in genres_file.readlines()]
    directory = config.dataset_dir + 'output/sentiment_word_texts/'
    doc_list = []
    genre_list = reduce_genres(pre_genre_list)
    print(pre_genre_list)
    print(genre_list)
    # create a list of lists containing all tokens contained in the text of a certain genre
    for genre in genre_list:
        book_list = []
        genre = genre.replace('/', ' ')
        books_of_genre = data.loc[data['genre'] == genre]
        
        for book in books_of_genre['filename']:
            book_list.append(book)
        
        genre_document = tfidf.genre_document(book_list, directory)
        doc_list.append(genre_document)

#     create index
    index = tfidf.create_index(genre_list, doc_list)
    # create tf_matrix
    tf_matrix = tfidf.create_tf_matrix(genre_list, doc_list)
    
    # create scores for each genre
    for genre, document in zip(genre_list, doc_list):
        genre = genre.replace('/', ' ')
        score_dict = {}
        document = set(document)
        try:
            
                for term in document:

                    score = tfidf.tfidf(term, genre, doc_list, index, tf_matrix)
                    score_dict[term] = score

                scores_file = open(config.dataset_dir +'output/top200_per_genre/' + genre + '.txt', 'w')

                for w in sorted(score_dict, key=score_dict.get, reverse=True):

                    scores_file.write('%s/n' % w)

                scores_file.close()

                print('success')

                if plot_wc:
                    font_path = config.dataset_dir + 'Open_Sans_Condensed/OpenSansCondensed-Light.ttf'
                    create_wordcloud(score_dict, genre)

        except ZeroDivisionError:
            print('reaallly')
            continue
        except ValueError:
            continue
    

tfidf_dict_per_genre = tfidf_per_genre(plot_wc=True)

['Science fiction fandom', 'Diary and Novel', 'Fantasy literature', 'War', 'Epistolary novel', 'Künstlerroman', 'Short stories', 'Legal thriller', 'Bush poetry', 'Fantasy novel', 'Sketch story', 'Romance novel', 'Christian literature', 'Autobiographical novel', 'Business', 'Poem', 'Planetary romance', 'Utopian fiction', 'Romantic novel', 'Young-adult fiction', 'Non-fiction', 'Poetry', 'Historical', 'unknown', 'Nature writing', 'Non-Fiction', 'Christian fiction', 'Tragicomedy', 'Detective stories', "Children's Literature", 'Encyclopedia', 'Thriller (genre)', 'Christian novel', 'Fairytale fantasy', 'Satirical', 'Christian Apologetics', 'Crime fiction', 'Prose poetry', 'Young adult fiction', 'Philosophy', 'Humor', 'Historical fantasy', 'Detective fiction', 'Hindi', 'Biography in literature', 'Decadent movement', 'Cookbook', 'Verse novel', 'Apocalyptic and post-apocalyptic fiction', 'Textbook', 'Essay', 'History of Ideas', 'Adventure novel', 'Family saga', 'Mystery novel', 'Short story', '

In [ ]:
list(tfidf_dict_per_genre.keys())[:4]

In [ ]:
len(list(tfidf_dict_per_genre.keys()))

In [ ]:
len(tfidf_dict_per_genre['Diary and Novel']) # may differ per genre

In [ ]:
n_words_per_genre = 100

In [ ]:
sample = tfidf_dict_per_genre['War']
i = list(sample.keys())[-1]
sample[i]

In [ ]:
max(list(sample.values()))

In [ ]:
sample

## Generate labels file

In [1]:
import pandas, os
import data, config
from utils import io

Using TensorFlow backend.


 Dataset :: namedtuple(
  'info': pandas.df
  'labels': pandas.df('filename.txt': 'genre')
  'genres': ['genre'] # unique genres
  'label_dataset': SubDataset
  'sentiment_dataset': SubDataset
  'book_sentiment_words_list': ['filename']

 SubDataset :: namedtuple(
   'dict_index_to_label' = dict to convert label_index -> label_name
   'dict_label_to_index'= dict to convert label_name -> label_index



In [2]:
info = pandas.read_csv(config.info_file)
book_list = os.listdir(config.sentiment_words_dir)[:]
labels = data.extract_genres(info, book_list)

In [3]:
labels

{'1005.txt': ['christian'],
 '1024.txt': ['adventur'],
 '1027.txt': ['western'],
 '103.txt': ['travel'],
 '1051.txt': ['comic'],
 '1056.txt': ['knstlerroman'],
 '106.txt': ['adventur'],
 '1066-0.txt': ['children'],
 '1080.txt': ['satir'],
 '1081.txt': ['polit'],
 '111.txt': ['children'],
 '113.txt': ['children'],
 '1142.txt': ['young adult'],
 '1153.txt': ['science fiction'],
 '1156.txt': ['satir'],
 '1160.txt': ['detect'],
 '1164.txt': ['dystopian'],
 '1188.txt': ['horror'],
 '1195.txt': ['children'],
 '1198.txt': ['adventur'],
 '12.txt': 'unknown',
 '1202.txt': ['frame stori'],
 '1207.txt': ['histor'],
 '1228.txt': ['science'],
 '1231.txt': ['cryptozoolog'],
 '1245.txt': ['romance'],
 '1257-0.txt': ['histor'],
 '1257-8.txt': ['histor'],
 '1257.txt': ['histor'],
 '1258.txt': ['romantic'],
 '1259-0.txt': ['romantic'],
 '1268.txt': ['adventur'],
 '13.txt': ['nonsens poetri'],
 '1343.txt': ['nonfict'],
 '1349-8.txt': ['adventur'],
 '1349.txt': ['adventur'],
 '1353-8.txt': ['science ficti

In [4]:
io.save_dict_to_csv(config.dataset_dir, 'labels', labels)

'../datasets/labels.csv'

# (oud)
Choose to most important to be kept in the feature-vector

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
import data, config, tfidf

In [ ]:
directory = config.dataset_dir + 'output/sentiment_word_texts'
book_list = os.listdir(directory)
book_list = book_list[:20]

In [ ]:
index = tfidf.create_index(directory, book_list)
tf_matrix = tfidf.create_tf_matrix(directory, book_list)

In [ ]:
tfidf_dict = tfidf.perform_tfidf(directory, book_list, index, tf_matrix)

In [ ]:
# (optional) show the result
w = WordCloud(background_color='white', width=900, height=500, 
                      max_words=1628,relative_scaling=1,normalize_plurals=False)
wordcloud = w.generate_from_frequencies(tfidf_dict)

In [ ]:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
# plt.savefig(config.dataset_dir + 'output/wordclouds/' + genre + '.png')

In [ ]:
# tfidf_dict_per_genre = wordcloud_per_genre()